# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
!pip install cassandra-driver

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/jovyan/work/data_modeling_cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
        CREATE  KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION = { 
          'class' : 'SimpleStrategy', 'replication_factor' : 1 
        }
    """)
except Exception as ex:
    print("Error creating keyspace.", ex)
    

#### Set Keyspace

In [8]:
try:
    session.set_keyspace("udacity")
except Exception as ex:
    print("Error setting keyspace.", ex)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS query1 (
            session_id int,
            item_in_session int,
            artist text,
            song_title text,
            song_length decimal,
            PRIMARY KEY (session_id, item_in_session)
        )
    """)
except Exception as ex:
    print("Error creating table.", ex)


                    

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO query1 (session_id, item_in_session, artist, song_title, song_length) VALUES "
        query = query + "(%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## Add in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute("SELECT artist, song_title, song_length FROM query1 WHERE session_id=338 and item_in_session=4")
    for row in rows:
        print(row.artist, row.song_title, row.song_length)
except Exception as ex:
    print("Error executing select.", ex)

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS query2 (
            user_id int,
            session_id int,
            item_in_session int,
            artist text,
            song_title text,
            user_first_name text,
            user_last_name text,
            PRIMARY KEY ((user_id, session_id), item_in_session)
        ) 
    """)
except Exception as ex:
    print("Error creating table.", ex)

                    

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO query2 (user_id, session_id, item_in_session, artist, song_title, user_first_name, user_last_name) VALUES "
        query = query + "(%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [14]:
## Add in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute("SELECT artist, song_title, user_first_name, user_last_name FROM query2 WHERE user_id=10 and session_id=182 ORDER BY item_in_session")
    for row in rows:
        print(row.artist, row.song_title, row.user_first_name, row.user_last_name)
except Exception as ex:
    print("Error executing select.", ex)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [15]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS query3 (
            song_title text,
            user_id int,
            user_first_name text,
            user_last_name text,
            PRIMARY KEY (song_title, user_id)
        ) 
    """)
except Exception as ex:
    print("Error creating table.", ex)

                    

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO query3 (song_title, user_id, user_first_name, user_last_name) VALUES "
        query = query + "(%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [17]:
## Add in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute("SELECT user_first_name, user_last_name FROM query3 WHERE song_title = 'All Hands Against His Own'")
    for row in rows:
        print(row.user_first_name, row.user_last_name)
except Exception as ex:
    print("Error executing select.", ex)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
## Drop the table before closing out the sessions
try:
    session.execute("DROP TABLE IF EXISTS query1");
    session.execute("DROP TABLE IF EXISTS query2");
    session.execute("DROP TABLE IF EXISTS query3");
except Exception as ex:
    print("Error droping tables. ", ex)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()